In [1]:
from pyspark import SparkConf, SparkContext

from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('kmeans').getOrCreate()

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,application_1607754714477_0001,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
%%configure -f
{
    "conf":{
        "spark.pyspark.python":"python",
        "spark.pyspark.virtualenv.enabled": "true",
        "spark.pyspark.virtualenv.type":"native",
        "spark.pyspark.virtualenv.bin.path": "/usr/bin/virtualenv"
    }
}

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
1,application_1607754714477_0002,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
1,application_1607754714477_0002,pyspark,idle,Link,Link,✔


In [18]:
# sc.install_pypi_package('pandas')
# sc.install_pypi_package('s3fs')
# sc.install_pypi_package('time')
# sc.install_pypi_package('matplotlib')
# sc.install_pypi_package('six==1.12')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
from pyspark.sql.types import IntegerType, FloatType
from pyspark.sql.functions import col
from pyspark.ml.clustering import KMeans
from pyspark.ml.evaluation import ClusteringEvaluator
#import matplotlib.pyplot as plt
from pyspark.ml.feature import VectorAssembler
import numpy as np
import pandas as pd

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Clustering Based on Countries Data

In [63]:
world_data = pd.read_csv('s3://bigdata678bdalvi17/time_series_covid_19_confirmed.csv')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [64]:
world_data.head()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

  Province/State Country/Region       Lat  ...  11/13/20  11/14/20  11/15/20
0            NaN    Afghanistan  33.93911  ...     42969     43035     43240
1            NaN        Albania  41.15330  ...     26701     27233     27830
2            NaN        Algeria  28.03390  ...     65975     66819     67679
3            NaN        Andorra  42.50630  ...      5725      5725      5872
4            NaN         Angola -11.20270  ...     13228     13374     13451

[5 rows x 303 columns]

In [65]:
world_data = world_data.drop(['Province/State', 'Lat', 'Long'], axis = 1)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [66]:
world_data = world_data.set_index('Country/Region')
world_data.head()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

                1/22/20  1/23/20  1/24/20  ...  11/13/20  11/14/20  11/15/20
Country/Region                             ...                              
Afghanistan           0        0        0  ...     42969     43035     43240
Albania               0        0        0  ...     26701     27233     27830
Algeria               0        0        0  ...     65975     66819     67679
Andorra               0        0        0  ...      5725      5725      5872
Angola                0        0        0  ...     13228     13374     13451

[5 rows x 299 columns]

In [67]:
world_sp = spark.createDataFrame(world_data)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [68]:
vecAssembler = VectorAssembler(inputCols=world_sp.columns, outputCol="features")
world_df = vecAssembler.transform(world_sp)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

##### Computing the Cost Function, to build a cost curve
##### Determining K value using Elbow rule and Silhouette Score

In [11]:
import time
start_time = time.time()
# computing the cost function for best value of K
cost_world = np.zeros(20)
for k in range(2,20):
    kmeans = KMeans().setK(k).setSeed(1).setFeaturesCol("features")
    model = kmeans.fit(world_df.sample(False,0.1, seed=42))
    cost_world[k] = model.computeCost(world_df)
end_time = time.time()

print(end_time - start_time)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

52.111921072

In [12]:

cost_ = pd.DataFrame(cost_world)
cost_ = cost_[2:][:]
x = range(2,20)
cost_['Value of K'] = x
cost_['Cost Value'] = cost_[:][0]
cost_ = cost_.drop([0], axis = 1)
cost_

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

    Value of K    Cost Value
2            2  1.027137e+16
3            3  1.017248e+16
4            4  1.001429e+16
5            5  9.983494e+15
6            6  9.978587e+15
7            7  9.977637e+15
8            8  9.976452e+15
9            9  9.976380e+15
10          10  9.976314e+15
11          11  9.975943e+15
12          12  9.975848e+15
13          13  9.975816e+15
14          14  9.975811e+15
15          15  9.975792e+15
16          16  9.975789e+15
17          17  9.975789e+15
18          18  9.975704e+15
19          19  9.975704e+15

In [69]:
evaluator = ClusteringEvaluator()
import time
start_time = time.time()

silhoutte_state = np.zeros(20)
for k in range(2,20):
    kmeans = KMeans().setK(k).setSeed(1).setFeaturesCol("features")
    model = kmeans.fit(world_df.select('features')).transform(world_df)
    silhoutte_state[k] = evaluator.evaluate(model)
end_time = time.time()

print(end_time - start_time)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

33.4959549904

In [71]:
score = pd.DataFrame(silhoutte_state)
score = score[2:][:]
x = range(2,20)
score['Value of K'] = x
score['Silhoutte Score'] = score[:][0]
score = score.drop([0], axis = 1)
score

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

    Value of K  Silhoutte Score
2            2         0.996111
3            3         0.953069
4            4         0.952681
5            5         0.925043
6            6         0.928137
7            7         0.899465
8            8         0.870519
9            9         0.874056
10          10         0.884841
11          11         0.885043
12          12         0.843766
13          13         0.845644
14          14         0.845694
15          15         0.809009
16          16         0.803813
17          17         0.768143
18          18         0.811298
19          19         0.826571

In [70]:
import time
start_time = time.time()

kmeans = KMeans(k=4, seed=1)  # 4 clusters here
model = kmeans.fit(world_df.select('features'))

end_time = time.time()
print(end_time - start_time)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

22.7054820061

In [72]:
transformed = model.transform(world_df)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [73]:
silhouette = evaluator.evaluate(transformed)
silhouette

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

0.9526812731017638

In [74]:
result = transformed.toPandas()
x = world_data.index
result['Country'] = x

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [75]:
result0 = result.iloc[:,-2:]

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [76]:
cluster_1 = result0[result0['prediction'] == 0]
cluster_1

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

     prediction         Country
6             0       Argentina
55            0           Chile
89            0        Colombia
126           0          France
130           0         Germany
145           0            Iran
149           0           Italy
178           0          Mexico
203           0            Peru
209           0          Russia
226           0    South Africa
228           0           Spain
258           0  United Kingdom

In [77]:
cluster_2 = result0[result0['prediction'] == 1]
cluster_2

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

     prediction                 Country
0             1             Afghanistan
1             1                 Albania
2             1                 Algeria
3             1                 Andorra
4             1                  Angola
5             1     Antigua and Barbuda
7             1                 Armenia
8             1               Australia
9             1               Australia
10            1               Australia
11            1               Australia
12            1               Australia
13            1               Australia
14            1               Australia
15            1               Australia
16            1                 Austria
17            1              Azerbaijan
18            1                 Bahamas
19            1                 Bahrain
20            1              Bangladesh
21            1                Barbados
22            1                 Belarus
23            1                 Belgium
24            1                  Belize


In [78]:
cluster_3 = result0[result0['prediction'] == 2]
cluster_3

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

     prediction Country
244           2      US

In [79]:
cluster_4 = result0[result0['prediction'] == 3]
cluster_4

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

     prediction Country
30            3  Brazil
143           3   India

#### Clustering Analysis based on US Data

In [80]:
data = pd.read_csv('s3://bigdata678bdalvi17/Confirmed_US.csv')

state_df = data.drop(['UID', 'iso2', 'iso3', 'code3', 'FIPS', 'Admin2', 'Lat', 'Long_', 'Country_Region', 'Combined_Key'], axis=1)
province_df = data.drop(['UID', 'iso2', 'iso3', 'code3', 'FIPS', 'Admin2', 'Lat', 'Long_', 'Country_Region', 'Province_State'], axis=1)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [81]:
province_df = province_df.set_index('Combined_Key')
province_df.head()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

                      1/22/2020  1/23/2020  ...  11/14/2020  11/15/2020
Combined_Key                                ...                        
Autauga, Alabama, US          0          0  ...        2435        2456
Baldwin, Alabama, US          0          0  ...        7596        7646
Barbour, Alabama, US          0          0  ...        1123        1128
Bibb, Alabama, US             0          0  ...         978         986
Blount, Alabama, US           0          0  ...        2518        2549

[5 rows x 299 columns]

In [82]:
state_df = state_df.groupby(['Province_State']).mean()
state_df.head()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

                1/22/2020  1/23/2020  ...    11/14/2020    11/15/2020
Province_State                        ...                            
Alabama               0.0        0.0  ...   3128.159420   3156.840580
Alaska                0.0        0.0  ...    746.870968    768.193548
American Samoa        0.0        0.0  ...      0.000000      0.000000
Arizona               0.0        0.0  ...  16061.941176  16202.117647
Arkansas              0.0        0.0  ...   1716.441558   1727.792208

[5 rows x 299 columns]

In [83]:
state_sp = spark.createDataFrame(state_df)
province_sp = spark.createDataFrame(province_df)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [84]:
#state_sp = state_sp.select(*(col(c).cast("float").alias(c) for c in df.columns[1:]))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

#### Clustering based on States and Districts

In [91]:
vecAssembler = VectorAssembler(inputCols=state_sp.columns, outputCol="features")
new_df = vecAssembler.transform(state_sp)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [86]:
# computing the cost function for best value of K

start_time = time.time()

cost = np.zeros(20)
for k in range(2,20):
    kmeans = KMeans().setK(k).setSeed(1).setFeaturesCol("features")
    model = kmeans.fit(new_df.sample(False,0.1, seed=42))
    cost[k] = model.computeCost(new_df)
    
end_time = time.time()
print(end_time - start_time)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

34.8633761406

In [87]:
cost_ = pd.DataFrame(cost)
cost_ = cost_[2:][:]
x = range(2,20)
cost_['Value of K'] = x
cost_['Cost Value'] = cost_[:][0]
cost_ = cost_.drop([0], axis = 1)
cost_

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

    Value of K    Cost Value
2            2  2.985686e+10
3            3  1.552945e+10
4            4  1.307620e+10
5            5  1.219122e+10
6            6  1.215141e+10
7            7  1.215141e+10
8            8  1.215141e+10
9            9  1.215141e+10
10          10  1.215141e+10
11          11  1.215141e+10
12          12  1.215141e+10
13          13  1.215141e+10
14          14  1.215141e+10
15          15  1.215141e+10
16          16  1.215141e+10
17          17  1.215141e+10
18          18  1.215141e+10
19          19  1.215141e+10

In [62]:
evaluator = ClusteringEvaluator()
import time
start_time = time.time()

silhoutte_state = np.zeros(20)
for k in range(2,20):
    kmeans = KMeans().setK(k).setSeed(1).setFeaturesCol("features")
    model = kmeans.fit(new_df.select('features')).transform(new_df)
    silhoutte_state[k] = evaluator.evaluate(model)
end_time = time.time()

print(end_time - start_time)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

24.3839437962

In [36]:
score = pd.DataFrame(silhoutte_state)
score = score[2:][:]
x = range(2,20)
score['Value of K'] = x
score['Silhoutte Score'] = score[:][0]
score = score.drop([0], axis = 1)
score

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

    Value of K  Silhoutte Score
2            2         0.906496
3            3         0.836518
4            4         0.816028
5            5         0.763590
6            6         0.734720
7            7         0.744758
8            8         0.774522
9            9         0.753595
10          10         0.687081
11          11         0.722012
12          12         0.627821
13          13         0.713211
14          14         0.615755
15          15         0.676286
16          16         0.599689
17          17         0.529569
18          18         0.600021
19          19         0.606389

In [92]:
import time
start_time = time.time()

kmeans = KMeans(k=4, seed=1)  # 4 clusters here
model = kmeans.fit(new_df.select('features'))

end_time =  time.time() 
print(end_time - start_time)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

0.704065084457

In [93]:
transformed = model.transform(new_df)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [95]:
# silhouette = evaluator.evaluate(transformed)
# silhouette

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [96]:
result = transformed.toPandas()
x = state_df.index
result['State'] = x

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [97]:
result0 = result.iloc[:,-2:]

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [107]:
cluster_1 = result0[result0['prediction'] == 0]
cluster_1

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

    prediction                 State
0            0               Alabama
8            0              Delaware
10           0  District of Columbia
12           0               Georgia
14           0                  Guam
17           0              Illinois
22           0             Louisiana
24           0              Maryland
26           0              Michigan
32           0                Nevada
37           0        North Carolina
40           0                  Ohio
43           0          Pennsylvania
45           0          Rhode Island
46           0        South Carolina
48           0             Tennessee
49           0                 Texas
50           0                  Utah
54           0            Washington
56           0             Wisconsin

In [98]:
cluster_2 = result0[result0['prediction'] == 1]
cluster_2

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

    prediction       State
3            1     Arizona
5            1  California
11           1     Florida

In [99]:
cluster_3 = result0[result0['prediction'] == 2]
cluster_3

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

    prediction          State
7            2    Connecticut
25           2  Massachusetts
34           2     New Jersey
36           2       New York

In [100]:
cluster_4 = result0[result0['prediction'] == 3]
cluster_4

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

    prediction                     State
1            3                    Alaska
2            3            American Samoa
4            3                  Arkansas
6            3                  Colorado
9            3          Diamond Princess
13           3            Grand Princess
15           3                    Hawaii
16           3                     Idaho
18           3                   Indiana
19           3                      Iowa
20           3                    Kansas
21           3                  Kentucky
23           3                     Maine
27           3                 Minnesota
28           3               Mississippi
29           3                  Missouri
30           3                   Montana
31           3                  Nebraska
33           3             New Hampshire
35           3                New Mexico
38           3              North Dakota
39           3  Northern Mariana Islands
41           3                  Oklahoma
42           3  

#### Clustering based on Counties (Provinces)

In [108]:
vecAssembler = VectorAssembler(inputCols=province_sp.columns, outputCol="features")
new_df_ = vecAssembler.transform(province_sp)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [109]:
import time
start_time = time.time()

# computing the cost function for best value of K
cost = np.zeros(20)
for k in range(2,20):
    kmeans = KMeans().setK(k).setSeed(1).setFeaturesCol("features")
    model = kmeans.fit(new_df_.sample(False,0.1, seed=42))
    cost[k] = model.computeCost(new_df_)
    
end_time = time.time()
print(end_time - start_time)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

48.5245220661

In [110]:
cost_ = pd.DataFrame(cost)
cost_ = cost_[2:][:]
x = range(2,20)
cost_['Value of K'] = x
cost_['Cost Value'] = cost_[:][0]
cost_ = cost_.drop([0], axis = 1)
cost_

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

    Value of K    Cost Value
2            2  1.854343e+13
3            3  1.328562e+13
4            4  1.241803e+13
5            5  1.186447e+13
6            6  8.959339e+12
7            7  8.885394e+12
8            8  8.623531e+12
9            9  8.488723e+12
10          10  8.547327e+12
11          11  8.547952e+12
12          12  8.481038e+12
13          13  8.288529e+12
14          14  8.315760e+12
15          15  8.269802e+12
16          16  8.210134e+12
17          17  8.232026e+12
18          18  8.211597e+12
19          19  8.194651e+12

In [111]:
import time
start_time = time.time()

kmeans = KMeans(k=6, seed=1)  # 5 clusters here
model = kmeans.fit(new_df_.select('features'))

end_time = time.time()
print(end_time - start_time)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

1.30310821533

In [112]:
transformed = model.transform(new_df_)
result_ = transformed.toPandas()
x = province_df.index
result_['State'] = x

result0_ = result_.iloc[:,-2:]

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [113]:
cluster_1 = result0_[result0_['prediction'] == 0]
cluster_1

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

      prediction                      State
0              0       Autauga, Alabama, US
1              0       Baldwin, Alabama, US
2              0       Barbour, Alabama, US
3              0          Bibb, Alabama, US
4              0        Blount, Alabama, US
5              0       Bullock, Alabama, US
6              0        Butler, Alabama, US
7              0       Calhoun, Alabama, US
8              0      Chambers, Alabama, US
9              0      Cherokee, Alabama, US
10             0       Chilton, Alabama, US
11             0       Choctaw, Alabama, US
12             0        Clarke, Alabama, US
13             0          Clay, Alabama, US
14             0      Cleburne, Alabama, US
15             0        Coffee, Alabama, US
16             0       Colbert, Alabama, US
17             0       Conecuh, Alabama, US
18             0         Coosa, Alabama, US
19             0     Covington, Alabama, US
20             0      Crenshaw, Alabama, US
21             0       Cullman, 

In [114]:
cluster_2 = result0_[result0_['prediction'] == 1]
cluster_2

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

     prediction                        State
213           1  Los Angeles, California, US

In [115]:
cluster_3 = result0_[result0_['prediction'] == 2]
cluster_3

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

      prediction                    State
108            2    Maricopa, Arizona, US
382            2  Miami-Dade, Florida, US
640            2       Cook, Illinois, US
2798           2        Harris, Texas, US

In [116]:
cluster_4 = result0_[result0_['prediction'] == 3]
cluster_4

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

      prediction                      State
228            3  Riverside, California, US
345            3       Broward, Florida, US
1813           3          Clark, Nevada, US
1902           3        Bronx, New York, US
1923           3        Kings, New York, US
1929           3       Nassau, New York, US
1941           3       Queens, New York, US
1952           3      Suffolk, New York, US
2754           3          Dallas, Texas, US

In [117]:
cluster_5 = result0_[result0_['prediction'] == 4]
cluster_5

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

      prediction                            State
112            4                Pima, Arizona, US
204            4           Fresno, California, US
209            4             Kern, California, US
224            4           Orange, California, US
231            4   San Bernardino, California, US
232            4        San Diego, California, US
321            4       Fairfield, Connecticut, US
354            4               Duval, Florida, US
367            4        Hillsborough, Florida, US
387            4              Orange, Florida, US
390            4          Palm Beach, Florida, US
392            4            Pinellas, Florida, US
468            4              Fulton, Georgia, US
475            4            Gwinnett, Georgia, US
777            4              Marion, Indiana, US
1252           4         Montgomery, Maryland, US
1254           4    Prince George's, Maryland, US
1268           4         Essex, Massachusetts, US
1272           4     Middlesex, Massachusetts, US


In [118]:
cluster_6 = result0_[result0_['prediction'] == 5]
cluster_6

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

      prediction                          State
36             5         Jefferson, Alabama, US
44             5           Madison, Alabama, US
48             5            Mobile, Alabama, US
50             5        Montgomery, Alabama, US
63             5        Tuscaloosa, Alabama, US
110            5            Navajo, Arizona, US
113            5             Pinal, Arizona, US
117            5              Yuma, Arizona, US
121            5           Benton, Arkansas, US
178            5          Pulaski, Arkansas, US
191            5       Washington, Arkansas, US
195            5        Alameda, California, US
201            5   Contra Costa, California, US
207            5       Imperial, California, US
210            5          Kings, California, US
215            5          Marin, California, US
218            5         Merced, California, US
221            5       Monterey, California, US
229            5     Sacramento, California, US
233            5  San Francisco, Califor